In [2]:
import ee
import datetime
import time # To potentially add delays if needed, though usually not necessary

# --- Authentication/Initialization ---
try:
    ee.Initialize()
except Exception as e:
    print(f"Earth Engine initialization failed: {e}")
    print("Attempting authentication...")
    ee.Authenticate() # Trigger the authentication flow
    ee.Initialize() # Try initializing again after authentication

In [3]:
import ee
import datetime
import time # Optional, for potential delays

def download_country_nightlights():
    """
    Downloads monthly VIIRS nightlight imagery for a user-selected country.
    - Shows the list of available countries.
    - Accepts input for country number or name.
    - Filters the data for the chosen country.
    - Exports images to a country-specific folder in Google Drive.
    """

    # --- Authentication/Initialization ---
    try:
        print("INFO: Initializing Earth Engine...")
        ee.Initialize()
        print("INFO: Earth Engine Initialized Successfully.")
    except Exception as e:
        print(f"ERROR: Earth Engine initialization failed: {e}")
        print("INFO: Attempting authentication...")
        try:
            ee.Authenticate()
            ee.Initialize()
            print("INFO: Earth Engine Re-Initialized Successfully after authentication.")
        except Exception as auth_e:
            print(f"ERROR: Earth Engine authentication and initialization failed: {auth_e}")
            return # Cannot proceed

    # --- Load GAUL Level 0 (Country) Feature Collection ---
    try:
        print("INFO: Loading country boundaries (FAO/GAUL/2015/level0)...")
        gaul_level0 = ee.FeatureCollection("FAO/GAUL/2015/level0")
        print("INFO: Country boundaries loaded.")
    except Exception as e:
        print(f"ERROR: Failed to load country boundaries: {e}")
        return

    # --- Get and Display Country List ---
    print("INFO: Fetching list of available countries...")
    try:
        country_names_raw = gaul_level0.aggregate_array('ADM0_NAME').getInfo()
        if not country_names_raw:
            print("ERROR: Could not retrieve country names.")
            return
        # Create a unique, sorted list for display
        country_list_display = sorted(list(set(country_names_raw)))
        print(f"INFO: Found {len(country_list_display)} unique countries.")
    except ee.EEException as e:
        print(f"ERROR: Failed fetching country list names from GEE: {e}")
        return
    except Exception as e:
        print(f"ERROR: An unexpected error occurred fetching country names: {e}")
        return

    print("\n--- Available Countries ---")
    for i, country in enumerate(country_list_display):
        print(f"{i+1}. {country}")
    print("---------------------------\n")

    # --- User Country Selection ---
    selected_country_name = None
    while True:
        choice = input("Enter country number or name: ").strip()
        try:
            # Try interpreting input as a number
            country_index = int(choice) - 1
            if 0 <= country_index < len(country_list_display):
                selected_country_name = country_list_display[country_index]
                print(f"INFO: Selected country by number: {selected_country_name}")
                break
            else:
                print("WARNING: Invalid country number. Please try again.")
        except ValueError:
            # Input is not a number, try interpreting as a name
            # Check exact match first
            if choice in country_list_display:
                 selected_country_name = choice
                 print(f"INFO: Selected country by name: {selected_country_name}")
                 break
            else:
                 # Case-insensitive check
                 found_match = False
                 for name in country_list_display:
                     if name.lower() == choice.lower():
                         selected_country_name = name # Use the correct case from the list
                         print(f"INFO: Interpreted '{choice}' as '{selected_country_name}'.")
                         found_match = True
                         break
                 if found_match:
                     break
                 else:
                     print(f"WARNING: Invalid country name '{choice}'. Please try again.")

    # --- Define Region of Interest (ROI) based on selection ---
    print(f"INFO: Filtering boundaries for {selected_country_name}...")
    try:
        # Filter the collection server-side by the selected name
        country_roi_feature = gaul_level0.filter(ee.Filter.eq('ADM0_NAME', selected_country_name)).first()

        # Verify that the feature was found (important!)
        if not country_roi_feature.getInfo(): # Small getInfo call on the feature possible here
             print(f"ERROR: Could not find the feature for '{selected_country_name}' after filtering. Check GAUL data or spelling.")
             # You might want to loop back to selection or exit
             return
        else:
            print(f"INFO: Found boundary feature for {selected_country_name}.")
            # print("ROI Geometry Type:", country_roi_feature.geometry().getInfo()['type']) # Optional check

    except ee.EEException as e:
        print(f"ERROR: GEE error while filtering for {selected_country_name}: {e}")
        return
    except Exception as e:
        print(f"ERROR: Unexpected error while filtering for {selected_country_name}: {e}")
        return

    # --- Define Date Range (Entire VIIRS Monthly Availability) ---
    start_date = '2012-04-01'
    end_date = datetime.datetime.now().strftime('%Y-%m-%d')
    print(f"INFO: Fetching monthly images for {selected_country_name} from {start_date} up to {end_date}")

    # --- Define Image Collection ---
    print("INFO: Defining VIIRS image collection and applying filters...")
    try:
        viirs_collection = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG") \
            .select('avg_rad') \
            .filterDate(start_date, end_date) \
            .filterBounds(country_roi_feature.geometry()) # Filter by the selected country's geometry
        print("INFO: VIIRS collection defined and filtered.")
    except ee.EEException as e:
        print(f"ERROR: GEE error defining or filtering VIIRS collection: {e}")
        return
    except Exception as e:
        print(f"ERROR: Unexpected error defining or filtering VIIRS collection: {e}")
        return


    # --- Function to Process and Export a Single Image ---
    def export_nightlight_image(image, roi_feature, folder_name, country_name_safe):
        """Clips and exports a single night light image for the selected country."""
        img_id_str = 'unknown_id'
        date_str = 'unknown_date'
        try:
            # Get image date for filename
            img_date = ee.Date(image.get('system:time_start'))
            date_str = img_date.format('YYYY_MM').getInfo()
            img_id_str = image.id().getInfo() # Get ID for logging

            # Define export parameters using selected country name
            description = f'{country_name_safe}_VIIRS_{date_str}'
            file_name = description # filename will be description + '.tif'

            # Clip the image using the country's geometry
            clipped_image = image.clip(roi_feature.geometry())

            # Define the export task
            task = ee.batch.Export.image.toDrive(
                image=clipped_image,
                description=description,
                folder=folder_name,      # Use the dynamic folder name
                fileNamePrefix=file_name,# Use the dynamic file name prefix
                region=roi_feature.geometry(), # Use the geometry of the feature
                scale=500,                  # VIIRS monthly resolution is ~500m
                crs='EPSG:4326',            # Standard Lat/Lon projection
                maxPixels=2e10              # Increased maxPixels for larger areas
            )

            task.start()
            # print(f"INFO: Export task started for {description}") # Can be noisy

        except ee.EEException as e:
            print(f"ERROR: GEE Error processing/exporting image ID {img_id_str} (Date: {date_str}): {e}")
        except Exception as e:
            print(f"ERROR: General Error processing/exporting image ID {img_id_str} (Date: {date_str}): {e}")

    # --- Iterate Through the Filtered Images and Export ---
    # Create a safe name for folders/files (replace spaces, etc.)
    safe_country_name = selected_country_name.replace(' ', '_').replace('/', '_').replace('\\', '_')
    export_folder = f'GEE_NightLights_{safe_country_name}' # Dynamic folder name

    print(f"INFO: Preparing to export images to Google Drive folder: '{export_folder}'")

    try:
        image_list = viirs_collection.toList(viirs_collection.size())
        num_images = image_list.size().getInfo()
    except ee.EEException as e:
        print(f"\nERROR: GEE Error getting the final image list or size: {e}")
        print("INFO: This might happen for very large countries or date ranges.")
        return
    except Exception as e:
        print(f"\nERROR: Unexpected error retrieving final image list: {e}")
        return

    if num_images == 0:
        print(f"INFO: No VIIRS monthly images found for {selected_country_name} in the specified period ({start_date} to {end_date}).")
    else:
        print(f"INFO: Found {num_images} monthly images for {selected_country_name}. Starting export tasks...")
        tasks_initiated = 0
        for i in range(num_images):
            try:
                image = ee.Image(image_list.get(i))
                export_nightlight_image(image, country_roi_feature, export_folder, safe_country_name)
                tasks_initiated += 1
                # Optional: Add a small delay every N tasks if hitting limits
                # if (i + 1) % 20 == 0:
                #    print(f"INFO: Initiated {tasks_initiated}/{num_images} tasks. Pausing briefly...")
                #    time.sleep(5)

            except ee.EEException as e:
                 print(f"ERROR: GEE Error getting/processing image at index {i} from list: {e}")
            except Exception as e:
                 print(f"ERROR: Unexpected error processing image at index {i}: {e}")

        print(f"\nINFO: Finished initiating {tasks_initiated} export tasks.")
        if tasks_initiated != num_images:
             print(f"WARNING: Expected {num_images} tasks, but only initiated {tasks_initiated}. Some might have failed.")

    print(f"INFO: Check the 'Tasks' tab in GEE (https://code.earthengine.google.com/tasks) or your Google Drive folder '{export_folder}'. Exports may take time.")
    print("INFO: Process finished.")

# --- Run the Function ---
if __name__ == "__main__":
    download_country_nightlights()

INFO: Initializing Earth Engine...
INFO: Earth Engine Initialized Successfully.
INFO: Loading country boundaries (FAO/GAUL/2015/level0)...
INFO: Country boundaries loaded.
INFO: Fetching list of available countries...
INFO: Found 276 unique countries.

--- Available Countries ---
1. Abyei
2. Afghanistan
3. Aksai Chin
4. Albania
5. Algeria
6. American Samoa
7. Andorra
8. Angola
9. Anguilla
10. Antarctica
11. Antigua and Barbuda
12. Argentina
13. Armenia
14. Aruba
15. Arunachal Pradesh
16. Ashmore and Cartier Islands
17. Australia
18. Austria
19. Azerbaijan
20. Azores Islands
21. Bahamas
22. Bahrain
23. Baker Island
24. Bangladesh
25. Barbados
26. Bassas da India
27. Belarus
28. Belgium
29. Belize
30. Benin
31. Bermuda
32. Bhutan
33. Bird Island
34. Bolivia
35. Bosnia and Herzegovina
36. Botswana
37. Bouvet Island
38. Brazil
39. British Indian Ocean Territory
40. British Virgin Islands
41. Brunei Darussalam
42. Bulgaria
43. Burkina Faso
44. Burundi
45. Cambodia
46. Cameroon
47. Canada
48